In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from torch import nn
import torch.nn.functional as F
from tensorflow.keras import layers, Model, optimizers
from tensorflow.keras.layers import LayerNormalization,Input
from torch.utils.data import Dataset, DataLoader
train = pd.read_csv('/kaggle/input/bengaliai-speech/train.csv')
train['path']  = ['/kaggle/input/bengaliai-speech/train_mp3s/'+str(i)+'.mp3' for i in train['id']]

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
class BengaliData(Dataset):
    def __init__(self,train):
        # Initialize your data, download, etc.
        self.train = train
    
    def __len__(self):
        return len(self.data_paths['path'])

    def __getitem__(self, index):
        obs_path = self.train['path'][index]
        data = np.load(obs_path+'/data.npy')
        labels = np.load(obs_path+'/labels.npy')
        labels = labels.transpose(2, 0, 1)
        labels = torch.from_numpy(labels)
        labels = labels.float()
        return data, labels

In [3]:
path = train['path'][1]
import torchaudio

# Load an MP3 file into a tensor
waveform, sample_rate = torchaudio.load(path)

# Print the shape of the waveform tensor
print(waveform.shape)

torch.Size([1, 78336])


for i in range(10000):
    path = train['path'][i] 
    waveform, sample_rate = torchaudio.load(path)
    file_name = '/kaggle/working/time_testing/'+str(train['id'][i])+'.pt'
    torch.save(waveform, file_name)

In [29]:
import time

start_time = time.time()

for i in range(10000):
    file_name = '/kaggle/working/time_testing/'+str(train['id'][i])+'.pt'
    y = torch.load(file_name)

end_time = time.time()

print(f'Time taken: {end_time - start_time:.2f} seconds')

Time taken: 5.08 seconds


In [44]:
import librosa

# Load an audio file as a waveform
waveform, sample_rate = librosa.load(path = train['path'][2])

# Extract MFCCs from the waveform
mfccs = librosa.feature.mfcc(y=waveform, sr=sample_rate)

# Print the shape of the MFCCs tensor
print(mfccs.shape)

(20, 204)
